# Homework — Week 3: Factor Models & Return Decomposition

> *"By one count, over 400 factors have been proposed to explain stock returns — more factors than months of out-of-sample data to test them."*

## Your Mission

This week you build the infrastructure that every cross-sectional strategy rests on.

In the lecture, we showed you *why* factor models matter — from CAPM's elegant failures to the factor zoo's overfitting epidemic. In the seminar, you tested individual pieces: replicating a single factor, running one Fama-MacBeth regression, decomposing one portfolio. Now you scale up.

You will build three things:

1. **A Factor Factory** — a complete pipeline that constructs all six Fama-French factors from raw data and validates them against Ken French's official benchmarks. This is the factory floor of quantitative finance: the data infrastructure that AQR, Dimensional, and every systematic fund builds before anything else.

2. **A Cross-Sectional Feature Matrix** — a reusable system that transforms messy fundamental and price data into a clean, standardized panel ready for ML consumption. This is the bridge between factor research (this week) and ML-based alpha models (Week 4). The feature matrix you build here is the *direct input* to next week's gradient boosting and neural network models.

3. **A Factor Model Horse Race** — a head-to-head comparison of CAPM, FF3, and FF5 using Fama-MacBeth regressions on your full stock universe. You will quantify exactly how much cross-sectional variation each model explains, which factors are genuinely priced, and — most importantly — how much *residual alpha* remains unexplained. That residual is the opportunity space for ML.

By the end, you will understand something that takes many junior quants months on the job to grasp: factor models explain only about 8–15% of cross-sectional return variation. The remaining 85%+ is noise *and* signal, mixed together. Week 4's ML models will try to separate them — but only if the feature matrix you build here is clean.

## Deliverables

1. **The Factor Factory** — `FactorBuilder` class producing 6 monthly factor return series (Mkt-RF, SMB, HML, RMW, CMA, MOM), validated against official Ken French data.
2. **The Cross-Sectional Feature Matrix** — `FeatureEngineer` class producing a standardized panel of 7 features (4 fundamental + 3 technical), saved to Parquet for Week 4.
3. **The Factor Model Horse Race** — Fama-MacBeth regressions for CAPM, FF3, and FF5 with R-squared progression, significance tests, and residual alpha analysis.

In [ ]:
%pip install yfinance getfactormodels linearmodels statsmodels matplotlib numpy pandas tqdm -q

import time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
from linearmodels import FamaMacBeth as LMFamaMacBeth
from tqdm import tqdm

plt.rcParams.update({
    "figure.figsize": (12, 6),
    "axes.grid": True,
    "grid.alpha": 0.3,
    "font.size": 11,
})

## Data Download

Everything in this homework starts from the same raw materials: 200 S&P 500 stocks spanning every sector, daily prices from 2014–2024, fundamental data from yfinance, and official factor returns from Ken French's data library. We download everything once here and work from memory throughout.

A word of warning: the fundamental download loops through 200 tickers sequentially, hitting yfinance's API for balance sheets, income statements, and company info. This takes about 2–3 minutes. If you are on a slow connection or yfinance throttles you, some tickers may fail — the code handles this gracefully and moves on. We only need about 150+ tickers to have enough cross-sectional breadth for all three deliverables.

In [ ]:
# ── Ticker universe: 200 S&P 500 stocks across all sectors ──
TICKERS = [
    # Technology
    "AAPL", "MSFT", "NVDA", "AVGO", "AMD", "CSCO", "ADBE", "CRM", "INTC", "TXN",
    "QCOM", "AMAT", "LRCX", "MU", "NOW", "INTU", "SNPS", "CDNS", "KLAC", "MCHP",
    # Communication Services
    "META", "GOOG", "NFLX", "DIS", "CMCSA", "T", "VZ", "TMUS", "EA", "TTWO",
    # Consumer Discretionary
    "AMZN", "TSLA", "HD", "MCD", "NKE", "LOW", "SBUX", "TJX", "BKNG", "ORLY",
    "MAR", "YUM", "DHI", "CMG", "ROST", "LEN", "GPC", "BBY", "POOL", "GRMN",
    # Consumer Staples
    "PG", "KO", "PEP", "COST", "WMT", "PM", "MO", "CL", "KMB", "GIS",
    "SJM", "HSY", "KHC", "MNST", "STZ", "KR", "TSN", "SYY", "CHD", "MKC",
    # Energy
    "XOM", "CVX", "COP", "SLB", "EOG", "MPC", "PSX", "VLO", "OXY", "HES",
    "DVN", "FANG", "HAL", "BKR", "CTRA", "APA", "TRGP", "KMI", "WMB", "OKE",
    # Financials
    "JPM", "BAC", "WFC", "GS", "MS", "BLK", "SCHW", "AXP", "USB", "PNC",
    "TFC", "AIG", "MET", "PRU", "ALL", "CB", "MMC", "AON", "ICE", "CME",
    # Healthcare
    "LLY", "UNH", "JNJ", "PFE", "ABT", "MRK", "TMO", "AMGN", "MDT", "ISRG",
    "BMY", "GILD", "VRTX", "REGN", "ZTS", "SYK", "BSX", "BDX", "EW", "HCA",
    # Industrials
    "CAT", "GE", "HON", "UNP", "RTX", "BA", "DE", "LMT", "MMM", "GD",
    "WM", "ITW", "EMR", "ETN", "FDX", "CSX", "NSC", "PCAR", "ROK", "FAST",
    # Materials
    "LIN", "APD", "SHW", "ECL", "NEM", "FCX", "NUE", "VMC", "MLM", "DOW",
    # Real Estate
    "PLD", "AMT", "CCI", "EQIX", "PSA", "SPG", "DLR", "O", "WELL", "ARE",
    # Utilities
    "NEE", "DUK", "SO", "D", "AEP", "SRE", "EXC", "XEL", "WEC", "ES",
]

START = "2014-01-01"
END = "2024-12-31"

# --- Equity prices ---
print("Downloading equity prices (200 tickers, 2014-2024)...")
raw = yf.download(TICKERS, start=START, end=END, auto_adjust=True, progress=True)
prices = raw["Close"]
completeness = prices.notna().mean()
valid_tickers = completeness[completeness > 0.50].index.tolist()
prices = prices[valid_tickers]
prices.index = pd.to_datetime(prices.index).tz_localize(None)
print(f"Tickers downloaded: {len(valid_tickers)}/{len(TICKERS)}")

# Monthly returns
monthly_prices = prices.resample("ME").last()
monthly_returns = monthly_prices.pct_change().dropna(how="all")
print(f"Monthly returns: {monthly_returns.shape}")

Now for the fundamental data. This is the slow part — we loop through each ticker to grab balance sheets, income statements, sector, and market cap from yfinance. The retry logic handles transient API failures, and we track which tickers fall back to Net Income when Operating Income is unavailable (about 4% of tickers, mostly financials like JPM and GS that use a different income statement structure).

In [ ]:
# --- Fundamentals ---
bs_records, inc_records, shares_records = [], [], []
sector_map, mcap_map = {}, {}
failures = []

print(f"Downloading fundamentals for {len(valid_tickers)} tickers...")
for ticker_str in tqdm(valid_tickers, desc="Fundamentals"):
    for attempt in range(3):
        try:
            tk = yf.Ticker(ticker_str)
            bs = tk.balance_sheet
            if bs is not None and not bs.empty:
                for col_date in bs.columns:
                    row = {}
                    for field in ["Stockholders Equity", "Total Assets",
                                  "Ordinary Shares Number"]:
                        if field in bs.index:
                            row[field] = bs.loc[field, col_date]
                    if row:
                        row["ticker"] = ticker_str
                        row["date"] = pd.Timestamp(col_date).tz_localize(None)
                        bs_records.append(row)
                    if "Ordinary Shares Number" in bs.index:
                        shares_records.append({
                            "ticker": ticker_str,
                            "date": pd.Timestamp(col_date).tz_localize(None),
                            "shares": bs.loc["Ordinary Shares Number", col_date],
                        })
            inc = tk.income_stmt
            if inc is not None and not inc.empty:
                for col_date in inc.columns:
                    row = {"ticker": ticker_str,
                           "date": pd.Timestamp(col_date).tz_localize(None)}
                    for field in ["Operating Income", "Net Income",
                                  "Total Revenue", "Pretax Income"]:
                        if field in inc.index:
                            row[field] = inc.loc[field, col_date]
                    inc_records.append(row)
            info = tk.info
            sector_map[ticker_str] = info.get("sector", "Other")
            mcap_map[ticker_str] = info.get("marketCap", np.nan)
            break
        except Exception as e:
            if attempt < 2:
                time.sleep(2 ** (attempt + 1))
            else:
                failures.append(ticker_str)
                print(f"  FAILED {ticker_str}: {e}")

bs_df = pd.DataFrame(bs_records)
if not bs_df.empty:
    bs_df = bs_df.set_index(["ticker", "date"]).sort_index()
inc_df = pd.DataFrame(inc_records)
if not inc_df.empty:
    inc_df = inc_df.set_index(["ticker", "date"]).sort_index()
shares_df = pd.DataFrame(shares_records)
if not shares_df.empty:
    shares_df = shares_df.set_index(["ticker", "date"]).sort_index()
sector_s = pd.Series(sector_map, name="sector")
mcap_s = pd.Series(mcap_map, name="market_cap")

fundamentals = {
    "balance_sheet": bs_df,
    "income_stmt": inc_df,
    "sector": sector_s,
    "market_cap": mcap_s,
    "shares": shares_df,
}

print(f"Balance sheet records: {len(bs_df)}")
print(f"Income statement records: {len(inc_df)}")
print(f"Sectors: {sector_s.nunique()} unique")
print(f"Failures: {len(failures)}")

Finally, official factor returns from Ken French's data library. These are the benchmark we will validate our self-built factors against. The `getfactormodels` library wraps Ken French's public data and returns it in a clean format.

In [ ]:
from getfactormodels import FamaFrenchFactors, CarhartFactors

# FF5 factors
ff5_raw = FamaFrenchFactors(model="5", frequency="M")
ff5 = ff5_raw.to_pandas()
if "date" in ff5.columns:
    ff5["date"] = pd.to_datetime(ff5["date"])
    ff5 = ff5.set_index("date")
ff5.index = pd.to_datetime(ff5.index)
ff5.index.name = "date"

# FF3 (subset of FF5)
ff3 = ff5[["Mkt-RF", "SMB", "HML", "RF"]].copy()

# Carhart (includes MOM)
cf_raw = CarhartFactors(frequency="M")
carhart = cf_raw.to_pandas()
if "date" in carhart.columns:
    carhart["date"] = pd.to_datetime(carhart["date"])
    carhart = carhart.set_index("date")
carhart.index = pd.to_datetime(carhart.index)
carhart.index.name = "date"

print(f"FF5 factors: {list(ff5.columns)}, {len(ff5)} months")
print(f"Carhart factors: {list(carhart.columns)}, {len(carhart)} months")

---

# Deliverable 1: The Factor Factory

Your first mission: build a complete factor construction pipeline from scratch.

Every systematic fund on the planet starts here. Before you can run a backtest, train a model, or pitch a strategy, you need factor returns — the elemental building blocks of cross-sectional finance. At AQR, Dimensional, and Bridgewater, the team that maintains the factor construction pipeline is foundational infrastructure, like the data engineering team at a tech company.

You will build a `FactorBuilder` class that takes raw price and fundamental data and produces monthly return series for all six canonical factors: the market (Mkt-RF), size (SMB), value (HML), profitability (RMW), investment (CMA), and momentum (MOM). The class then validates its output against Ken French's official data and reports correlation and tracking error for each factor.

Here is the crucial twist that makes this more than a coding exercise: our universe is 200 S&P 500 stocks — large-caps only. Ken French's factors are built from the entire NYSE/AMEX/NASDAQ universe of 4,000–6,000 stocks, including genuine small-caps with market caps of \$100M–\$2B. Our "small" stocks have market caps above \$10B. This structural mismatch means some factors will replicate well and others will not, and the *hierarchy* of replication quality tells you something important about which characteristics have meaningful dispersion within large-caps and which require the full market universe.

### Student Workspace

Build a `FactorBuilder` class with these capabilities:
- Download raw data and compute fundamental characteristics (book-to-market, profitability, investment)
- Perform 2×3 double-sort portfolio formation (size median × characteristic terciles)
- Compute value-weighted monthly returns for each portfolio
- Construct long-short factor returns (SMB, HML, RMW, CMA, MOM, Mkt-RF)
- Validate against Ken French official data (correlation, tracking error)
- Produce a quality report (missing data, fallback tickers, portfolio composition)

Tease: the validation results will reveal a clear hierarchy — some factors replicate well with our data, others do not. The reason *why* each factor fails or succeeds is itself the insight.

In [ ]:
# TODO: Define the FactorBuilder class
# - __init__: accept monthly_returns, prices, fundamentals
# - compute_characteristics: extract B/M, profitability, investment from fundamentals
# - _double_sort: perform 2 x 3 sort on size and a characteristic

In [ ]:
# TODO: Add the portfolio return computation and factor construction methods
# - _compute_vw_returns: value-weighted monthly returns per portfolio
# - _build_long_short: SMB and characteristic factor from portfolio returns
# - build_all_factors: orchestrate the full 6-factor pipeline

Once the class skeleton is in place, the factory needs to actually *produce* something. The value-weighted return computation is where most subtle bugs hide — getting the weight normalization wrong silently breaks every downstream factor return. Think carefully about how to handle months where some tickers in a portfolio have missing returns.

In [ ]:
# TODO: Add the validation method
# - validate: compare each self-built factor against official Ken French data
# - report correlation, tracking error, number of overlapping months

With the pipeline built, the final step is execution and validation. Running the builder end-to-end should produce 6 factor return series spanning ~131 months. Validation against Ken French's official data is not optional — it is how you know whether your construction methodology is correct or subtly broken.

In [ ]:
# TODO: Run the pipeline end-to-end
# builder = FactorBuilder(monthly_returns, prices, fundamentals)
# factor_returns = builder.build_all_factors()

In [ ]:
# TODO: Validate and visualize
# validation = builder.validate(ff5, carhart)
# Plot cumulative factor returns

---
## ━━━ SOLUTION: Deliverable 1 ━━━

The `FactorBuilder` class below encapsulates the full pipeline. A few design choices worth noting: the class computes characteristics *once* from the most recent fundamentals and applies those fixed assignments over the entire price window. This is a **static sort** — a deliberate simplification because yfinance provides only ~4–5 annual fundamental periods per ticker (~2021–2025), making proper annual rebalancing over 2014–2024 impossible. The proper Fama-French methodology rebalances portfolios each June using the prior fiscal year's fundamentals. Our static approach introduces look-ahead bias — a stock classified as "value" using its 2025 book equity may have been "growth" in 2014. We will see how this affects validation results shortly.

In [ ]:
class FactorBuilder:
    """Construct Fama-French 5 factors + momentum from raw data.

    Attributes:
        monthly_returns: DataFrame of monthly returns (date x ticker).
        fundamentals: Dict of fundamental DataFrames from data_setup.
        factor_returns: DataFrame of monthly self-built factor returns.
        quality_report: Dict summarizing data quality issues.
    """

    def __init__(self, monthly_returns, prices, fundamentals):
        self.monthly_returns = monthly_returns
        self.prices = prices
        self.fundamentals = fundamentals
        self.factor_returns = None
        self.quality_report = {
            "missing_equity": [],
            "missing_income": [],
            "net_income_fallback": [],
            "portfolio_counts": {},
        }

    def compute_characteristics(self):
        """Compute book-to-market, profitability, and investment."""
        bs = self.fundamentals["balance_sheet"]
        inc = self.fundamentals["income_stmt"]
        mcap = self.fundamentals["market_cap"]

        records = []
        for ticker in self.monthly_returns.columns:
            row = {"ticker": ticker}

            # Book equity
            equity = np.nan
            if ticker in bs.index.get_level_values("ticker"):
                eq_vals = bs.loc[ticker].sort_index()["Stockholders Equity"].dropna()
                if len(eq_vals) > 0 and eq_vals.iloc[-1] > 0:
                    equity = eq_vals.iloc[-1]
                else:
                    self.quality_report["missing_equity"].append(ticker)
            else:
                self.quality_report["missing_equity"].append(ticker)

            # Market cap and B/M
            mk = mcap.get(ticker, np.nan)
            if not np.isnan(equity) and not np.isnan(mk) and mk > 0:
                row["book_to_market"] = equity / mk
                row["market_cap"] = mk

            # Operating profitability (OP / equity)
            if ticker in inc.index.get_level_values("ticker"):
                tk_inc = inc.loc[ticker].sort_index()
                oi = tk_inc.get("Operating Income", pd.Series(dtype=float)).dropna()
                if len(oi) == 0:
                    ni = tk_inc.get("Net Income", pd.Series(dtype=float)).dropna()
                    if len(ni) > 0:
                        oi = ni
                        self.quality_report["net_income_fallback"].append(ticker)
                if len(oi) > 0 and not np.isnan(equity) and equity > 0:
                    row["profitability"] = oi.iloc[-1] / equity
            else:
                self.quality_report["missing_income"].append(ticker)

            # Investment (asset growth)
            if ticker in bs.index.get_level_values("ticker"):
                assets = bs.loc[ticker].sort_index()["Total Assets"].dropna()
                if len(assets) > 1 and assets.iloc[-2] > 0:
                    row["investment"] = (assets.iloc[-1] / assets.iloc[-2]) - 1

            records.append(row)

        self.chars = pd.DataFrame(records).set_index("ticker")
        return self.chars

    def _double_sort(self, char_col, n_groups=3):
        """Perform 2 x n_groups sort on size and a characteristic.

        Returns dict mapping portfolio name to list of tickers.
        """
        valid = self.chars.dropna(subset=["market_cap", char_col]).copy()

        # Size split: median
        size_median = valid["market_cap"].median()
        valid["size_group"] = np.where(
            valid["market_cap"] < size_median, "S", "B"
        )

        # Characteristic split
        breakpoints = [valid[char_col].quantile(q)
                       for q in np.linspace(0, 1, n_groups + 1)]

        def assign_char_group(val):
            for i in range(n_groups):
                if val <= breakpoints[i + 1]:
                    return i
            return n_groups - 1

        valid["char_group"] = valid[char_col].apply(assign_char_group)

        portfolios = {}
        for size in ["S", "B"]:
            for cg in range(n_groups):
                mask = (valid["size_group"] == size) & (valid["char_group"] == cg)
                pf_tickers = valid[mask].index.tolist()
                pf_name = f"{size}/{cg}"
                portfolios[pf_name] = pf_tickers

        return portfolios, valid

    def _compute_vw_returns(self, portfolios, valid_chars):
        """Compute value-weighted monthly returns for each portfolio."""
        pf_returns = {}

        for date in self.monthly_returns.index:
            rets = self.monthly_returns.loc[date].dropna()
            date_rets = {}

            for pf_name, tickers in portfolios.items():
                avail = [t for t in tickers if t in rets.index]
                if len(avail) == 0:
                    date_rets[pf_name] = np.nan
                    continue
                weights = valid_chars.loc[avail, "market_cap"]
                weights = weights / weights.sum()
                date_rets[pf_name] = (rets[avail] * weights).sum()

            pf_returns[date] = date_rets

        return pd.DataFrame(pf_returns).T

    def _build_long_short(self, pf_returns, n_groups=3):
        """Build SMB and characteristic-based long-short factor."""
        # Factor = avg(S portfolios) - avg(B portfolios)
        s_cols = [c for c in pf_returns.columns if c.startswith("S/")]
        b_cols = [c for c in pf_returns.columns if c.startswith("B/")]

        smb = pf_returns[s_cols].mean(axis=1) - pf_returns[b_cols].mean(axis=1)

        # Characteristic factor: high - low
        high = n_groups - 1
        high_cols = [f"S/{high}", f"B/{high}"]
        low_cols = ["S/0", "B/0"]
        char_factor = pf_returns[high_cols].mean(axis=1) - \
                      pf_returns[low_cols].mean(axis=1)

        return smb, char_factor

The `_double_sort` method implements the classic Fama-French portfolio construction: split on size at the median, split on the characteristic into terciles, and form 2 x 3 = 6 portfolios. The `_build_long_short` then constructs the factor as the average return of the high-characteristic portfolios minus the average return of the low-characteristic portfolios, averaged across size groups. This design — averaging across the size dimension — ensures that each factor captures the *pure* characteristic effect, not a size effect in disguise.

Now let's add the orchestration method that builds all six factors, plus the validation method that compares our output to Ken French.

In [ ]:
def build_all_factors(self):
    """Build all 6 factors: Mkt-RF, SMB, HML, RMW, CMA, MOM."""
    self.compute_characteristics()

    factor_series = {}

    # Market factor (value-weighted universe excess return)
    common = self.monthly_returns.index.intersection(ff3.index)
    rf = ff3.loc[common, "RF"]

    # VW market return approximation
    valid_mcap = self.chars["market_cap"].dropna()
    weights = valid_mcap / valid_mcap.sum()
    mkt_tickers = valid_mcap.index.tolist()
    mkt_ret = (self.monthly_returns[mkt_tickers] * weights).sum(axis=1)
    factor_series["Mkt-RF"] = mkt_ret.loc[common] - rf

    # HML (book-to-market: high = value, low = growth)
    pf_bm, valid_bm = self._double_sort("book_to_market", n_groups=3)
    pf_bm_rets = self._compute_vw_returns(pf_bm, valid_bm)
    smb_bm, hml = self._build_long_short(pf_bm_rets)

    self.quality_report["portfolio_counts"]["HML"] = {
        k: len(v) for k, v in pf_bm.items()
    }

    # RMW (profitability: high = robust, low = weak)
    if "profitability" in self.chars.columns:
        valid_prof = self.chars.dropna(subset=["market_cap", "profitability"])
        if len(valid_prof) >= 30:
            pf_prof, valid_p = self._double_sort("profitability", n_groups=3)
            pf_prof_rets = self._compute_vw_returns(pf_prof, valid_p)
            smb_prof, rmw = self._build_long_short(pf_prof_rets)
            self.quality_report["portfolio_counts"]["RMW"] = {
                k: len(v) for k, v in pf_prof.items()
            }
        else:
            rmw = pd.Series(0, index=self.monthly_returns.index)
    else:
        rmw = pd.Series(0, index=self.monthly_returns.index)

    # CMA (investment: low growth = conservative, high growth = aggressive)
    # Note: CMA = Conservative - Aggressive, so we INVERT: low - high
    if "investment" in self.chars.columns:
        valid_inv = self.chars.dropna(subset=["market_cap", "investment"])
        if len(valid_inv) >= 30:
            pf_inv, valid_i = self._double_sort("investment", n_groups=3)
            pf_inv_rets = self._compute_vw_returns(pf_inv, valid_i)
            smb_inv, inv_factor = self._build_long_short(pf_inv_rets)
            cma = -inv_factor  # Invert: conservative minus aggressive
            self.quality_report["portfolio_counts"]["CMA"] = {
                k: len(v) for k, v in pf_inv.items()
            }
        else:
            cma = pd.Series(0, index=self.monthly_returns.index)
    else:
        cma = pd.Series(0, index=self.monthly_returns.index)

    # SMB: average of SMB from all three sorts
    smb_components = [smb_bm]
    if "profitability" in self.chars.columns:
        smb_components.append(smb_prof)
    if "investment" in self.chars.columns:
        smb_components.append(smb_inv)
    smb = pd.concat(smb_components, axis=1).mean(axis=1)

    # Momentum (12-1 month return)
    mom_monthly = []
    for date in self.monthly_returns.index:
        mom_end = date - pd.DateOffset(months=1)
        mom_start = date - pd.DateOffset(months=12)
        mask = (self.prices.index >= mom_start) & \
               (self.prices.index <= mom_end)
        if mask.sum() < 20:
            continue
        mom_prices = self.prices.loc[mask]
        if len(mom_prices) < 2:
            continue
        momentum = (mom_prices.iloc[-1] / mom_prices.iloc[0]) - 1
        momentum = momentum.dropna()

        rets = self.monthly_returns.loc[date].dropna()
        common_t = momentum.index.intersection(rets.index)
        if len(common_t) < 30:
            continue

        mom_sorted = momentum[common_t].sort_values()
        n = len(mom_sorted)
        losers = mom_sorted.iloc[:int(n * 0.3)].index
        winners = mom_sorted.iloc[int(n * 0.7):].index

        # Value-weighted
        if "market_cap" in self.chars.columns:
            w_win = self.chars.loc[
                self.chars.index.isin(winners), "market_cap"
            ].dropna()
            w_los = self.chars.loc[
                self.chars.index.isin(losers), "market_cap"
            ].dropna()
            if len(w_win) > 0 and len(w_los) > 0:
                w_win = w_win / w_win.sum()
                w_los = w_los / w_los.sum()
                win_ret = (rets[w_win.index] * w_win).sum()
                los_ret = (rets[w_los.index] * w_los).sum()
            else:
                win_ret = rets[winners].mean()
                los_ret = rets[losers].mean()
        else:
            win_ret = rets[winners].mean()
            los_ret = rets[losers].mean()

        mom_monthly.append({"date": date, "MOM": win_ret - los_ret})

    mom_df = pd.DataFrame(mom_monthly).set_index("date")["MOM"]

    # Combine all factors
    self.factor_returns = pd.DataFrame({
        "Mkt-RF": factor_series["Mkt-RF"],
        "SMB": smb,
        "HML": hml,
        "RMW": rmw,
        "CMA": cma,
        "MOM": mom_df,
    })

    return self.factor_returns

FactorBuilder.build_all_factors = build_all_factors

Notice the CMA inversion: `cma = -inv_factor`. The Fama-French convention is "Conservative Minus Aggressive" — low-investment firms minus high-investment firms. Our `_build_long_short` returns high minus low by default, so we flip the sign. This is the kind of subtle convention that causes silent bugs in production factor pipelines. If you get CMA's sign wrong, your factor loadings invert, and your risk decomposition tells a backwards story.

Momentum is constructed differently from the fundamental-based factors: it uses a 12-1 month price window (skipping the most recent month to avoid short-term reversal contamination) and does not require any fundamental data. This makes it our **cleanest benchmark** — if momentum replicates well, we know the price-based pipeline works correctly.

In [ ]:
def validate(self, official_ff5, official_carhart):
    """Compare self-built factors against official data."""
    results = {}
    factor_map = {
        "SMB": ("SMB", official_ff5),
        "HML": ("HML", official_ff5),
        "RMW": ("RMW", official_ff5),
        "CMA": ("CMA", official_ff5),
        "MOM": ("MOM", official_carhart),
    }

    for self_name, (off_name, off_df) in factor_map.items():
        if self_name not in self.factor_returns.columns:
            continue
        if off_name not in off_df.columns:
            continue

        self_s = self.factor_returns[self_name].dropna()
        common = self_s.index.intersection(off_df.index)
        if len(common) < 12:
            continue

        corr = self_s.loc[common].corr(off_df.loc[common, off_name])
        te = (self_s.loc[common] - off_df.loc[common, off_name]).std() \
             * np.sqrt(12)

        results[self_name] = {
            "correlation": corr,
            "tracking_error_ann": te,
            "n_months": len(common),
        }

    return results

FactorBuilder.validate = validate

With the class complete, let's run it end-to-end. The `build_all_factors` method orchestrates the full pipeline: compute characteristics from fundamentals, form double-sort portfolios for each factor, compute value-weighted returns, and construct long-short factor returns.

In [ ]:
builder = FactorBuilder(monthly_returns, prices, fundamentals)
factor_returns = builder.build_all_factors()

print(f"Factor returns shape: {factor_returns.shape}")
print(f"\nFactor return statistics (monthly):")
print(factor_returns.describe().round(6).to_string())

validation = builder.validate(ff5, carhart)

print("\nValidation Against Official Data:")
print(f"{'Factor':<8} {'Corr':>8} {'TE (ann)':>10} {'N months':>10}")
print("-" * 40)
for name, vals in validation.items():
    print(f"{name:<8} {vals['correlation']:>8.4f} "
          f"{vals['tracking_error_ann']:>10.4f} {vals['n_months']:>10}")

Six factor return series, each spanning roughly 131 months. Now the moment of truth: how well do our self-built factors match Ken French's official data? This validation step is not optional — at any serious quant shop, factor returns that have not been benchmarked against a known reference are not trusted, full stop.

The validation reveals a clear hierarchy. HML replicates best (r ~ 0.83), followed by MOM (r ~ 0.64), then RMW (r ~ 0.35), SMB (r ~ 0.33), and CMA (r ~ 0.29). Compare this to production benchmarks: Tidy Finance (Scheuch, Voigt & Weiss, 2023), using the full CRSP/Compustat universe with proper annual rebalancing, achieves correlations above 0.95 for *all* factors.

But the hierarchy itself is the insight. HML replicates well because value-growth dispersion exists within large-caps — some S&P 500 companies trade at 20x earnings (growth) while others trade at 8x (value). SMB replicates poorly (r ~ 0.33) because our "small" stocks have market caps above \$10B. Ken French's small-cap bucket includes companies at \$100M–\$2B. We are measuring large-versus-very-large, not small-versus-large.

An important caveat on the HML correlation: our implementation uses a **static sort** — we classify stocks once using ~2021–2025 fundamentals and apply those fixed assignments over the entire 2014–2024 price window. This introduces look-ahead bias and flatters the correlation relative to a proper annual-rebalancing approach. The high HML correlation is partly real (value/growth classifications are stable among large-cap S&P 500 stocks over a decade) and partly a methodological artifact.

Also note that MOM correlation is 0.64 here — lower than the 0.85 seen in the seminar. The difference is not a bug. The seminar used equal-weighted portfolios for momentum, while the `FactorBuilder` uses value-weighted portfolios (matching the Fama-French methodology). Value-weighting concentrates the portfolio in mega-cap stocks, whose momentum signals are noisier relative to the broad market. This 0.21 correlation gap from a single weighting choice illustrates how "the momentum factor" is not a unique object — construction methodology matters.

In [ ]:
qr = builder.quality_report
print(f"\nQuality Report:")
print(f"  Missing equity: {len(qr['missing_equity'])} tickers")
print(f"  Missing income: {len(qr['missing_income'])} tickers")
print(f"  Net Income fallback: {len(qr['net_income_fallback'])} tickers")
print(f"  Portfolio counts:")
for factor_name, counts in qr["portfolio_counts"].items():
    print(f"    {factor_name}: {counts}")

The quality report tells us the cost of using free data. About 11 tickers are missing book equity entirely (negative equity or no balance sheet data — often financials with complex capital structures). Another 15 tickers fall back to Net Income when Operating Income is unavailable. This is standard for yfinance: banks and financial companies like JPM and GS use a different income statement structure where "Operating Income" is not a meaningful line item. In production, Compustat standardizes these across all firms. With free data, you approximate.

The portfolio counts confirm that all six double-sort portfolios have at least 19 stocks each — adequate for value-weighted return computation. If any portfolio had fewer than 10 stocks, we would need to widen breakpoints or reduce the number of sorts.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
for col in factor_returns.columns:
    cum = (1 + factor_returns[col].dropna()).cumprod()
    ax.plot(cum.index, cum, label=col, lw=1.5)
ax.set(title="Self-Built Factor Cumulative Returns",
       xlabel="Date", ylabel="Cumulative Return")
ax.legend(ncol=3)
ax.grid(True, alpha=0.3)
ax.axhline(1.0, color="gray", lw=0.5, ls="--")
plt.tight_layout()
plt.show()

The cumulative return plot brings the validation numbers to life. The market factor (Mkt-RF) dominates with strong cumulative growth — reflecting the exceptional 2014–2024 equity bull market driven by tech. Momentum shows its characteristic pattern: steady accumulation punctuated by sharp drawdowns. SMB bounces around zero — no surprise, since our universe lacks true small-caps. HML and CMA are flat or slightly negative, consistent with the widely discussed "death of value" in the post-2007 era.

If you are a portfolio manager at a systematic fund, this plot is your morning dashboard. The factor you are loading on dictates your P&L. And the fact that different factors have radically different return profiles over the *same* period is why factor allocation — choosing *which* factors to tilt toward — is a job unto itself.

---

# Deliverable 2: The Cross-Sectional Feature Matrix

Your second mission: build the bridge between factor models and machine learning.

In the lecture, we showed you that factors are really just features with economic theory behind them. Now you build the feature engineering system that translates raw financial data into a clean, standardized panel ready for ML consumption. This is *exactly* the preprocessing pipeline that a data scientist at an asset management firm would build — the step that sits between raw fundamental/price data and the gradient boosting or neural network models of Week 4.

Gu, Kelly & Xiu (2020) — the paper that settled the debate about whether ML works for stock prediction — used 94 firm characteristics. We will build 7 (4 fundamental + 3 technical). The scale is different, but the methodology is identical: compute raw characteristics, winsorize cross-sectionally, standardize to z-scores, and output a clean panel. The *how* matters more than the *how many*, because every one of the 94 features in GKX follows the same pipeline you are about to build.

A warning that will save you from a subtle trap: not all financial ratios are safe to z-score and feed to ML. P/E ratio — seemingly the most basic valuation metric — is pathological. It is undefined when earnings are zero, sign-flips for loss-making firms, and produces a non-monotonic cross-sectional distribution that breaks standardization. Earnings yield (E/P) is the correct feature because it is bounded and handles losses gracefully. This is why MSCI Barra models use E/P, not P/E, as a style factor.

### Student Workspace

Build a `FeatureEngineer` class that:
- Computes fundamental features: P/B, ROE, asset growth, earnings yield
- Computes technical features: 12-1 month momentum, 1-month reversal, 60-day trailing volatility
- Applies two-stage outlier control: [1st, 99th] percentile winsorization + z-score hard cap at ±3.0
- Provides both z-scored and rank-transformed outputs
- Reports missing data statistics
- Outputs an ML-ready panel saved to Parquet

Tease: the standardization verification will reveal why a z-cap at ±3 is necessary even after winsorization — with only ~179 stocks per cross-section, percentile clipping is not enough to tame heavy-tailed financial ratios.

In [ ]:
# TODO: Define the FeatureEngineer class
# - __init__: accept prices, monthly_returns, fundamentals
# - _compute_fundamental_chars: P/B, ROE, asset growth, earnings yield
# - build: assemble the full panel with technical features

In [ ]:
# TODO: Implement winsorization + z-scoring + rank transform
# - _winsorize_and_zscore: two-stage outlier control
# - _build_quality_report: missing data stats

With the class defined and the standardization logic in place, you are ready to run the pipeline. The `build` method will loop through every month in the sample, compute technical features (momentum, reversal, volatility), merge with static fundamentals, and apply winsorization + z-scoring cross-sectionally. Expect this to take 30–60 seconds for ~130 months.

In [ ]:
# TODO: Run the pipeline
# fe = FeatureEngineer(prices, monthly_returns, fundamentals)
# feature_matrix = fe.build()

After running the pipeline, you should have a panel with ~23,000 rows and 21 columns (7 raw + 7 z-scored + 7 rank-transformed). The final step is to extract the ML-ready subset (z-scored columns only), verify the standardization properties, and visualize the distributions.

In [ ]:
# TODO: Verify standardization and export ML-ready matrix
# ml_matrix = fe.get_ml_ready_matrix()
# ml_matrix.to_parquet("feature_matrix_ml.parquet")

In [ ]:
# TODO: Visualize feature distributions

---
## ━━━ SOLUTION: Deliverable 2 ━━━

The `FeatureEngineer` class follows the same pattern as `FactorBuilder`: compute raw characteristics, then process them cross-sectionally. The critical design choice is the two-stage outlier control. With only ~179 stocks per cross-section, the [1st, 99th] percentile winsorization clips only about 2 observations per tail — not enough to tame structurally skewed features like P/B, which can range from 1 to 200+. The hard z-cap at ±3.0 after standardization is the industry-standard safety net, used in production pipelines at Barra/MSCI and Axioma.

In [ ]:
class FeatureEngineer:
    """Construct a standardized cross-sectional feature matrix.

    Computes fundamental features (P/E, P/B, ROE, asset growth,
    earnings yield) and technical features (momentum, reversal,
    volatility), then applies winsorization and cross-sectional
    standardization.

    Attributes:
        feature_matrix: Panel DataFrame (MultiIndex: date, ticker)
            with standardized feature columns.
        quality_report: Dict with missing data statistics.
    """

    FUNDAMENTAL_FEATURES = [
        "pb_ratio", "roe", "asset_growth", "earnings_yield",
    ]
    TECHNICAL_FEATURES = [
        "momentum", "reversal", "volatility",
    ]
    ALL_FEATURES = FUNDAMENTAL_FEATURES + TECHNICAL_FEATURES

    def __init__(self, prices, monthly_returns, fundamentals):
        self.prices = prices
        self.monthly_returns = monthly_returns
        self.fundamentals = fundamentals
        self.feature_matrix = None
        self.quality_report = {}

    def _compute_fundamental_chars(self):
        """Compute fundamental ratios from balance sheet and income."""
        bs = self.fundamentals["balance_sheet"]
        inc = self.fundamentals["income_stmt"]
        mcap = self.fundamentals["market_cap"]

        records = {}
        for ticker in self.monthly_returns.columns:
            row = {}

            # Book equity and total assets
            equity = np.nan
            total_assets = np.nan
            prev_assets = np.nan

            if ticker in bs.index.get_level_values("ticker"):
                tk_bs = bs.loc[ticker].sort_index()
                eq = tk_bs["Stockholders Equity"].dropna()
                if len(eq) > 0 and eq.iloc[-1] > 0:
                    equity = eq.iloc[-1]
                assets = tk_bs["Total Assets"].dropna()
                if len(assets) > 0:
                    total_assets = assets.iloc[-1]
                if len(assets) > 1:
                    prev_assets = assets.iloc[-2]

            # Income items
            net_income = np.nan
            revenue = np.nan
            if ticker in inc.index.get_level_values("ticker"):
                tk_inc = inc.loc[ticker].sort_index()
                ni = tk_inc.get("Net Income", pd.Series(dtype=float)).dropna()
                if len(ni) > 0:
                    net_income = ni.iloc[-1]
                rev = tk_inc.get("Total Revenue", pd.Series(dtype=float)).dropna()
                if len(rev) > 0:
                    revenue = rev.iloc[-1]

            mk = mcap.get(ticker, np.nan)

            # P/B ratio
            if not np.isnan(mk) and mk > 0 and not np.isnan(equity) \
               and equity > 0:
                row["pb_ratio"] = mk / equity

            # ROE
            if not np.isnan(equity) and equity > 0 \
               and not np.isnan(net_income):
                row["roe"] = net_income / equity

            # Asset growth
            if not np.isnan(total_assets) and not np.isnan(prev_assets) \
               and prev_assets > 0:
                row["asset_growth"] = (total_assets / prev_assets) - 1

            # Earnings yield
            if not np.isnan(mk) and mk > 0 and not np.isnan(net_income):
                row["earnings_yield"] = net_income / mk

            records[ticker] = row

        return pd.DataFrame(records).T

Notice that we compute **earnings yield** (E/P) rather than P/E. P/E is the ratio every financial news outlet quotes, but it is a trap for ML practitioners. When a firm reports a loss, P/E becomes negative — but the cross-sectional ordering between a firm with P/E = -5 and P/E = +5 is meaningless (one is unprofitable, the other is cheap). Earnings yield handles this gracefully: a loss-making firm simply has negative E/P. The cross-sectional ordering is always monotonic in "profitability per dollar of market cap." This is exactly why MSCI Barra uses E/P, not P/E, as a style factor.

Now the `build` method, which assembles the full panel by computing technical features for each month and merging them with the static fundamental characteristics.

In [ ]:
def build(self):
    """Build the full feature matrix panel."""
    fund_chars = self._compute_fundamental_chars()
    daily_returns = self.prices.pct_change().dropna(how="all")

    panel_records = []
    for date in self.monthly_returns.index:
        # Momentum: 12-1 month return
        mom_end = date - pd.DateOffset(months=1)
        mom_start = date - pd.DateOffset(months=12)
        mask = (self.prices.index >= mom_start) & \
               (self.prices.index <= mom_end)
        if mask.sum() < 20:
            continue
        mom_prices = self.prices.loc[mask]
        if len(mom_prices) < 2:
            continue
        momentum = (mom_prices.iloc[-1] / mom_prices.iloc[0]) - 1

        # Short-term reversal (1-month return)
        reversal = self.monthly_returns.loc[date]

        # Trailing volatility (60-day)
        vol_idx = self.prices.index.get_indexer([date], method="pad")[0]
        vol_start = max(0, vol_idx - 60)
        recent = daily_returns.iloc[vol_start:vol_idx]
        volatility = recent.std() * np.sqrt(252)

        for ticker in self.monthly_returns.columns:
            if pd.isna(self.monthly_returns.loc[date, ticker]):
                continue

            row = {
                "date": date,
                "ticker": ticker,
                "momentum": momentum.get(ticker, np.nan),
                "reversal": reversal.get(ticker, np.nan),
                "volatility": volatility.get(ticker, np.nan),
            }

            # Add fundamental features
            if ticker in fund_chars.index:
                for col in self.FUNDAMENTAL_FEATURES:
                    if col in fund_chars.columns:
                        row[col] = fund_chars.loc[ticker, col]

            panel_records.append(row)

    panel = pd.DataFrame(panel_records)
    panel = panel.set_index(["date", "ticker"])

    # Winsorize and standardize
    self.feature_matrix = self._winsorize_and_zscore(panel)

    # Quality report
    self._build_quality_report()

    return self.feature_matrix

FeatureEngineer.build = build

The technical features — momentum, reversal, and volatility — are computed fresh for each month, while the fundamental features are static (computed once from the most recent annual filings). This is the standard approach: fundamentals update at most quarterly, while price-based signals update continuously. In production at a firm like Two Sigma or Citadel, the feature pipeline would re-compute price-based features daily and fundamental features quarterly, with point-in-time discipline to avoid look-ahead bias.

Now the winsorization and standardization machinery — the part that separates a clean feature matrix from one that will silently corrupt downstream ML models.

In [ ]:
def _winsorize_and_zscore(self, panel):
    """Winsorize at [1,99] percentiles, then z-score per month."""
    feature_cols = [c for c in self.ALL_FEATURES if c in panel.columns]

    def process_month(group):
        """Winsorize and z-score one month."""
        for col in feature_cols:
            vals = group[col].dropna()
            if len(vals) > 10:
                lo = vals.quantile(0.01)
                hi = vals.quantile(0.99)
                group[col] = group[col].clip(lo, hi)

                mean, std = group[col].mean(), group[col].std()
                if std > 0:
                    group[col + "_z"] = ((group[col] - mean) / std).clip(-3.0, 3.0)
                else:
                    group[col + "_z"] = 0.0

                # Rank transform (0 to 1)
                group[col + "_rank"] = group[col].rank(pct=True)

        return group

    result = panel.groupby(level="date", group_keys=False).apply(
        process_month
    )
    return result

FeatureEngineer._winsorize_and_zscore = _winsorize_and_zscore

Look at that `.clip(-3.0, 3.0)` on the z-scores. With ~179 stocks per cross-section, the [1st, 99th] percentile winsorization clips only about 2 observations per tail. For structurally skewed features — like P/B, where a handful of tech firms can have ratios above 50 while most firms sit between 1 and 10 — this is insufficient. The z-cap at ±3 is the industry-standard second stage: after percentile clipping in raw space, hard-cap in z-space. Barra USE4, Axioma, and every serious quant pipeline uses this two-stage approach.

The z-cap mechanically reduces the standard deviation of the heaviest-tailed features — you will see z-std values around 0.84–1.00 instead of the theoretical 1.00. This is the expected cost of tail truncation, not a bug. The small histogram spikes you will see at ±3 are the capped mass accumulating at the boundaries.

Now the quality report and the ML-ready export.

In [ ]:
def _build_quality_report(self):
    """Report missing data statistics."""
    fm = self.feature_matrix
    dates = fm.index.get_level_values("date").unique()

    for col in self.ALL_FEATURES:
        if col in fm.columns:
            missing_pct = fm[col].isna().mean()
            self.quality_report[col] = {
                "missing_pct": missing_pct,
                "n_non_null": fm[col].notna().sum(),
            }

    # Monthly missing rates
    worst_month_missing = {}
    for col in self.ALL_FEATURES:
        if col in fm.columns:
            monthly_missing = fm.groupby(level="date")[col].apply(
                lambda x: x.isna().mean()
            )
            worst_month_missing[col] = monthly_missing.max()
    self.quality_report["worst_month_missing"] = worst_month_missing

def get_ml_ready_matrix(self):
    """Return a clean panel for ML consumption.

    Drops rows with too many missing features and returns
    only z-scored columns.
    """
    z_cols = [c for c in self.feature_matrix.columns if c.endswith("_z")]
    clean = self.feature_matrix[z_cols].dropna(thresh=len(z_cols) - 2)
    return clean

FeatureEngineer._build_quality_report = _build_quality_report
FeatureEngineer.get_ml_ready_matrix = get_ml_ready_matrix

The `get_ml_ready_matrix` method applies a final filter: drop any row with more than 2 missing z-score features. This ensures that Week 4's models receive a panel where at least 5 of 7 features are present for every stock-month observation. The threshold of 2 is conservative — in production, you might impute the remaining NaNs with cross-sectional medians or use tree-based models that handle missing values natively.

Let's run the full pipeline.

In [ ]:
fe = FeatureEngineer(prices, monthly_returns, fundamentals)
feature_matrix = fe.build()

print(f"Feature matrix shape: {feature_matrix.shape}")
print(f"Columns: {feature_matrix.columns.tolist()}")

print(f"\nMissing Data Report:")
for feat in fe.ALL_FEATURES:
    if feat in fe.quality_report:
        info = fe.quality_report[feat]
        print(f"  {feat}: {info['missing_pct']:.1%} missing "
              f"({info['n_non_null']} non-null)")

print(f"\nWorst monthly missing rates:")
for feat, rate in fe.quality_report.get("worst_month_missing", {}).items():
    print(f"  {feat}: {rate:.1%}")

The panel has ~23,000 rows (approximately 179 tickers times 130 months) with raw features, z-scored features, and rank-transformed features for each of the 7 characteristics. Now let's check the quality report to see where the gaps are.

The pattern is clear: price-based features (momentum, reversal, volatility) have less than 1% missing data because they rely only on daily prices, which yfinance provides reliably. Fundamental features (pb_ratio, roe) have about 6% missing — those are the ~11 tickers with missing or negative book equity. Asset growth and earnings yield have slightly lower missing rates because they require less data (total assets is almost always available, and earnings yield uses Net Income which has broader coverage than Stockholders Equity).

Now let's verify the standardization is working correctly.

In [ ]:
z_cols = [c for c in feature_matrix.columns if c.endswith("_z")]
sample_dates = feature_matrix.index.get_level_values("date").unique()[:12]

z_means = []
z_stds = []
for date in sample_dates:
    month = feature_matrix.loc[date]
    for col in z_cols:
        vals = month[col].dropna()
        if len(vals) > 10:
            z_means.append(vals.mean())
            z_stds.append(vals.std())

print(f"\nZ-score verification (first 12 months):")
print(f"  Mean range: [{min(z_means):.4f}, {max(z_means):.4f}]")
print(f"  Std range: [{min(z_stds):.4f}, {max(z_stds):.4f}]")

Z-score means are near zero (within ±0.04) and standard deviations are in the range [0.84, 1.00]. The means are not exactly zero because the z-cap at ±3 slightly shifts the distribution — when you clip the right tail more than the left (or vice versa), the mean nudges toward the less-clipped side. The standard deviations below 1.0 reflect the mechanical compression from the z-cap: heavy-tailed features like pb_ratio lose more variance to the cap than well-behaved features like momentum. This is the expected cost of the two-stage outlier approach and is acceptable.

Let's export the ML-ready matrix and visualize the feature distributions.

In [ ]:
ml_matrix = fe.get_ml_ready_matrix()
print(f"\nML-ready matrix shape: {ml_matrix.shape}")
print(f"Columns: {ml_matrix.columns.tolist()}")

# Save for downstream use
ml_matrix.to_parquet("feature_matrix_ml.parquet")
print(f"Saved to: feature_matrix_ml.parquet")

The ML-ready matrix has ~23,000 rows and 7 z-scored feature columns — this is the direct input to Week 4's gradient boosting and neural network models. The Parquet format preserves the MultiIndex (date, ticker), column types, and is efficient to read. Every model you build next week will start with `pd.read_parquet("feature_matrix_ml.parquet")`.

Let's visualize the feature distributions to confirm the standardization and outlier control are working as intended.

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, col in enumerate(z_cols[:8]):
    ax = axes[i]
    vals = feature_matrix[col].dropna()
    ax.hist(vals, bins=50, alpha=0.7, edgecolor="k", linewidth=0.3)
    ax.set_title(col.replace("_z", ""), fontsize=10)
    ax.axvline(0, color="red", lw=1, ls="--")

plt.suptitle("Feature Distributions (Z-scored)", fontsize=12)
plt.tight_layout()
plt.show()

Notice the small spikes at ±3 in the heaviest-tailed features (pb_ratio, roe). Those are the observations that got capped by the z-score hard limit — the mass that would have been at z = 4, 5, or even 7+ without the cap is now piled up at the boundary. For well-behaved features like momentum and reversal, the distributions are smooth and roughly symmetric, with very little mass at the boundaries. This confirms that the two-stage outlier control is doing exactly what it should: taming the problematic features while leaving the well-behaved ones alone.

The feature correlation structure is also worth examining. P/B and ROE show a strong positive correlation (~0.85) — firms with high price-to-book tend to have high return on equity, because the market pays a premium for profitable companies. P/B and book-to-market show a strong negative correlation (~-0.65), which is expected since they are near-reciprocals (P/B = 1 / B/M). Momentum and reversal show a negative cross-sectional correlation (~-0.22), consistent with the well-documented short-term reversal effect: last month's winners tend to partially reverse.

---

# Deliverable 3: The Factor Model Horse Race

Your final mission: settle the question of which factor model best explains the cross-section of stock returns.

CAPM says one factor (beta) is enough. Fama and French said three factors are better (add size and value). Then they said five are better still (add profitability and investment). Each model claims to explain more of the cross-section — but how much more, exactly? And more importantly for your Week 4 ambitions: how much *residual alpha* remains unexplained? That residual is the signal that ML models will try to capture.

You will run a head-to-head Fama-MacBeth comparison of all three nested models on your full 200-stock universe. The Fama-MacBeth procedure — cross-sectional regressions every month, then time-series averages of the slopes — is the gold standard test for whether a characteristic carries a risk premium. It is the test that any quant researcher at AQR or Two Sigma would use to evaluate a new signal before committing capital.

Tease: the horse race delivers a clear winner — but a modest one. Even the best factor model leaves ~85% of cross-sectional return variation unexplained. The average stock has about 5% monthly unexplained return. That is both the promise and the curse of cross-sectional ML: there is enormous residual to capture, but most of it is noise.

### Student Workspace

Build the horse race:
1. Estimate rolling 60-month market betas for all stocks
2. Compute static fundamental characteristics (size, B/M, profitability, investment)
3. Build Fama-MacBeth panels for three nested models: CAPM (beta only), FF3 (+ size, value), FF5 (+ profitability, investment)
4. Run `linearmodels.FamaMacBeth` with Newey-West standard errors for each
5. Compare R-squared progression, factor significance, and residual alpha across models

In [ ]:
# TODO: Estimate rolling 60-month market betas
# For each month starting at month 60, regress each stock's
# excess returns on Mkt-RF over the trailing 60 months

In [ ]:
# TODO: Compute static characteristics
# log_mcap, book-to-market, profitability, investment

With betas and characteristics in hand, the next step is assembling the Fama-MacBeth panel and running all three regressions. Remember that `linearmodels.FamaMacBeth` requires a MultiIndex of (entity, time) and that you should standardize characteristics cross-sectionally before regression to make gamma coefficients comparable across variables.

In [ ]:
# TODO: Build the panel and run Fama-MacBeth for all three models
# CAPM: [beta]
# FF3: [beta, log_mcap, bm]
# FF5: [beta, log_mcap, bm, profitability, investment]

In [ ]:
# TODO: Compare R-squared, residual alpha, and visualize

---
## ━━━ SOLUTION: Deliverable 3 ━━━

The horse race begins with the slowest step: estimating rolling 60-month market betas for every stock in the universe. This is the time-series step of the Fama-MacBeth procedure — for each stock, we regress its excess returns on the market factor over a trailing window. The 60-month window is standard in academic finance (roughly matching a business cycle), though practitioners often use shorter windows (24–36 months) for more responsive beta estimates.

In [ ]:
common_idx = monthly_returns.index.intersection(ff3.index)
rf = ff3.loc[common_idx, "RF"]
excess_returns = monthly_returns.loc[common_idx].sub(rf, axis=0)
mkt_rf = ff3.loc[common_idx, "Mkt-RF"]

# Rolling 60-month market betas
window = 60
beta_panel = {}

for i in range(window, len(common_idx)):
    date = common_idx[i]
    mkt_window = mkt_rf.iloc[i - window:i]
    betas = {}
    for ticker in excess_returns.columns:
        y = excess_returns[ticker].iloc[i - window:i].dropna()
        if len(y) < 36:
            continue
        x = mkt_window.loc[y.index]
        X = sm.add_constant(x)
        model = sm.OLS(y, X).fit()
        betas[ticker] = model.params["Mkt-RF"]
    beta_panel[date] = betas

beta_df = pd.DataFrame(beta_panel).T
print(f"Rolling beta panel: {beta_df.shape}")

The rolling beta panel has one row per month (starting at month 61 — we need 60 months of history before the first estimate) and one column per ticker. This consumes the first ~5 years of our 11-year sample, leaving about 71 months of usable cross-sections. In production, you would estimate betas daily for faster response to regime changes, but monthly is standard for academic tests.

Now the static fundamental characteristics. These are the same characteristics we computed in Deliverable 1, but here they serve as Fama-MacBeth regressors rather than portfolio sort variables.

In [ ]:
# Static characteristics
bs = fundamentals["balance_sheet"]
inc = fundamentals["income_stmt"]
mcap = fundamentals["market_cap"]

fund_chars = {}
for ticker in monthly_returns.columns:
    chars = {}

    # Log market cap (size)
    if ticker in mcap.index and mcap[ticker] > 0:
        chars["log_mcap"] = np.log(mcap[ticker])

    # Book-to-market (value)
    if ticker in bs.index.get_level_values("ticker"):
        eq = bs.loc[ticker].sort_index()["Stockholders Equity"].dropna()
        if len(eq) > 0 and eq.iloc[-1] > 0:
            if ticker in mcap.index and mcap[ticker] > 0:
                chars["bm"] = eq.iloc[-1] / mcap[ticker]

    # Operating profitability
    if ticker in inc.index.get_level_values("ticker"):
        tk_inc = inc.loc[ticker].sort_index()
        oi = tk_inc.get("Operating Income", pd.Series(dtype=float)).dropna()
        if len(oi) == 0:
            oi = tk_inc.get("Net Income", pd.Series(dtype=float)).dropna()
        if len(oi) > 0:
            if ticker in bs.index.get_level_values("ticker"):
                eq2 = bs.loc[ticker].sort_index()["Stockholders Equity"].dropna()
                if len(eq2) > 0 and eq2.iloc[-1] > 0:
                    chars["profitability"] = oi.iloc[-1] / eq2.iloc[-1]

    # Investment (asset growth)
    if ticker in bs.index.get_level_values("ticker"):
        assets = bs.loc[ticker].sort_index()["Total Assets"].dropna()
        if len(assets) > 1 and assets.iloc[-2] > 0:
            chars["investment"] = (assets.iloc[-1] / assets.iloc[-2]) - 1

    fund_chars[ticker] = chars

fund_df = pd.DataFrame(fund_chars).T
print(f"Fundamental characteristics: {fund_df.shape}")

Now we assemble the panel that Fama-MacBeth will consume. The panel has one row per stock-month observation, with excess returns as the dependent variable and characteristics as independent variables. The key requirement for `linearmodels.FamaMacBeth` is a MultiIndex of (entity, time) — in our case, (ticker, date).

In [ ]:
# Build panels for each model specification
panel_records = []

for date in beta_df.index:
    for ticker in monthly_returns.columns:
        if date not in excess_returns.index:
            continue
        ret = excess_returns.loc[date, ticker]
        if pd.isna(ret):
            continue
        beta_val = beta_df.loc[date].get(ticker, np.nan)
        if pd.isna(beta_val):
            continue

        row = {
            "date": date,
            "ticker": ticker,
            "excess_ret": ret,
            "beta": beta_val,
        }
        if ticker in fund_df.index:
            for col in fund_df.columns:
                row[col] = fund_df.loc[ticker, col]
        panel_records.append(row)

full_panel = pd.DataFrame(panel_records)
full_panel = full_panel.set_index(["ticker", "date"]).sort_index()

all_chars = ["beta", "log_mcap", "bm", "profitability", "investment"]

def standardize_month(group):
    """Z-score standardize within each month."""
    for col in all_chars:
        if col in group.columns:
            vals = group[col]
            mean, std = vals.mean(), vals.std()
            if std > 0:
                group[col] = (vals - mean) / std
    return group

panel_std = full_panel.groupby(level="date", group_keys=False).apply(
    standardize_month
)

Before running Fama-MacBeth, we standardize all characteristics cross-sectionally within each month. This ensures that the risk premium estimates (gamma coefficients) are comparable across characteristics — otherwise, the coefficient for log_mcap (which ranges from ~23 to ~28) would be mechanically smaller than the coefficient for book-to-market (which ranges from ~0.01 to ~0.80) simply due to scale differences.

Now the main event: three nested Fama-MacBeth regressions. CAPM uses only beta. FF3 adds size (log market cap) and value (book-to-market). FF5 adds profitability and investment. For each model, we compute the average cross-sectional R-squared and the average absolute residual — the unexplained return that ML might capture.

The `linearmodels.FamaMacBeth` with `cov_type='kernel'` applies Newey-West standard errors, which correct for serial correlation in the time-series of cross-sectional slopes. Without this correction, t-statistics can be inflated — or deflated, depending on the autocorrelation structure.

In [ ]:
# Three model specifications
models = {
    "CAPM": ["beta"],
    "FF3": ["beta", "log_mcap", "bm"],
    "FF5": ["beta", "log_mcap", "bm", "profitability", "investment"],
}

fm_results = {}

for model_name, char_list in models.items():
    clean = panel_std.dropna(subset=char_list + ["excess_ret"])
    if len(clean) < 1000:
        print(f"  {model_name}: insufficient data ({len(clean)} obs)")
        continue

    dep = clean[["excess_ret"]]
    indep = clean[char_list]

    fm = LMFamaMacBeth(dep, indep).fit(cov_type="kernel")

    # Compute average cross-sectional R²
    r2_list = []
    dates = clean.index.get_level_values("date").unique()
    for date in dates:
        md = clean.loc[clean.index.get_level_values("date") == date]
        if len(md) < 30:
            continue
        y = md["excess_ret"].values
        X = sm.add_constant(md[char_list].values)
        model_ols = sm.OLS(y, X).fit()
        r2_list.append(model_ols.rsquared)

    avg_r2 = np.mean(r2_list)

    # Residual alpha
    resid_abs = []
    for date in dates:
        md = clean.loc[clean.index.get_level_values("date") == date]
        if len(md) < 30:
            continue
        y = md["excess_ret"].values
        X = sm.add_constant(md[char_list].values)
        model_ols = sm.OLS(y, X).fit()
        resid_abs.extend(np.abs(model_ols.resid))

    avg_abs_resid = np.mean(resid_abs)

    fm_results[model_name] = {
        "params": fm.params,
        "tstats": fm.tstats,
        "pvalues": fm.pvalues,
        "avg_r2": avg_r2,
        "avg_abs_residual": avg_abs_resid,
        "n_obs": len(clean),
        "n_months": len(dates),
    }

    print(f"\n{model_name} ({len(clean)} obs, {len(dates)} months):")
    print(f"  Avg cross-sectional R²: {avg_r2:.4f}")
    print(f"  Avg |residual|: {avg_abs_resid:.6f}")
    for char in char_list:
        print(f"  {char}: gamma={fm.params[char]:.6f}, "
              f"NW t={fm.tstats[char]:.2f}, p={fm.pvalues[char]:.4f}")

The results tell a clear story. R-squared increases monotonically: CAPM (~8.6%), FF3 (~13.9%), FF5 (~15.3%). Every factor you add improves the cross-sectional fit — but the improvements shrink: +5.4 percentage points from CAPM to FF3, only +1.4 from FF3 to FF5. This is diminishing returns in action: the first two factors (size and value) add more explanatory power than the next two (profitability and investment).

Look at which factors are statistically significant in the FF5 specification. Beta carries a significant premium (t ~ 2.37, p ~ 0.018) — surprising if you have read the textbook literature, which documents the "beta anomaly" (flat or negative cross-sectional relationship between beta and returns). But our sample is 2014–2024, dominated by the AI/tech boom where high-beta stocks (NVDA, TSLA, AMD) earned extraordinary returns. This is the same sample-dependence lesson from the lecture: the "right" answer to "is beta priced?" depends on which decade you examine.

Investment is also significant (t ~ 2.41) — firms with lower asset growth ("conservative" companies) earned higher risk-adjusted returns. Profitability is insignificant (t ~ -0.43), which is surprising given the literature. With CRSP data spanning 60+ years, profitability is one of the more robust factors (Novy-Marx, 2013). With our 71-month window and S&P 500 universe, there is not enough statistical power to detect it.

In [ ]:
r2_values = {name: res["avg_r2"] for name, res in fm_results.items()}
residuals = {name: res["avg_abs_residual"] for name, res in fm_results.items()}

print(f"\nR² Progression:")
for name in ["CAPM", "FF3", "FF5"]:
    if name in r2_values:
        print(f"  {name}: R² = {r2_values[name]:.4f}, "
              f"|resid| = {residuals[name]:.6f}")

The R-squared progression from 8.6% to 15.3% means that even the *best* factor model leaves about 85% of cross-sectional return variation unexplained. The average stock has roughly 4.9% monthly unexplained return (absolute residual) under FF5. To put that in perspective: if your ML model from Week 4 can predict even 1% of that 4.9% residual, you have a potentially tradeable signal. But the other 3.9% is noise that will fight you every step of the way.

This is the fundamental tension of cross-sectional ML: the opportunity space is enormous (85%+ unexplained variance), but the signal-to-noise ratio is terrible. This is why Gu, Kelly & Xiu (2020) found that neural networks improve upon linear models — but only by a modest margin. The noise floor is high.

Let's visualize the horse race.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# R² comparison
ax = axes[0]
model_names = list(fm_results.keys())
r2_vals = [fm_results[m]["avg_r2"] for m in model_names]
bars = ax.bar(model_names, r2_vals, color=["#66BB6A", "#42A5F5", "#EF5350"],
              edgecolor="k", linewidth=0.5)
for bar, val in zip(bars, r2_vals):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.001,
            f"{val:.4f}", ha="center", fontsize=10)
ax.set(title="Average Cross-Sectional R²", ylabel="R²")

# Residual alpha
ax = axes[1]
resid_vals = [fm_results[m]["avg_abs_residual"] * 100 for m in model_names]
bars = ax.bar(model_names, resid_vals, color=["#66BB6A", "#42A5F5", "#EF5350"],
              edgecolor="k", linewidth=0.5)
for bar, val in zip(bars, resid_vals):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05,
            f"{val:.2f}%", ha="center", fontsize=10)
ax.set(title="Average |Residual| (Monthly %)", ylabel="Absolute Residual (%)")

# t-statistics for FF5 factors
ax = axes[2]
if "FF5" in fm_results:
    ff5_chars = ["beta", "log_mcap", "bm", "profitability", "investment"]
    ff5_tstats = [fm_results["FF5"]["tstats"][c] for c in ff5_chars]
    colors = ["#2196F3" if abs(t) >= 2.0 else "#BDBDBD" for t in ff5_tstats]
    ax.barh(range(len(ff5_chars)), ff5_tstats, color=colors,
            edgecolor="k", linewidth=0.5)
    ax.set_yticks(range(len(ff5_chars)))
    ax.set_yticklabels(ff5_chars)
    ax.axvline(2.0, color="red", ls="--", lw=1)
    ax.axvline(-2.0, color="red", ls="--", lw=1)
    ax.set(title="FF5 Fama-MacBeth t-Statistics", xlabel="t-statistic")

plt.suptitle("Factor Model Horse Race: CAPM vs. FF3 vs. FF5", y=1.02)
plt.tight_layout()
plt.show()

The three-panel visualization makes the horse race results immediately legible. The left panel shows the R-squared staircase — each model improves on the last, but with diminishing steps. The middle panel shows the average absolute residual barely budging: from 5.05% (CAPM) to 4.91% (FF5). Going from one factor to five reduced the unexplained return by only 0.14 percentage points per month. The right panel shows which factors clear the significance bar (red dashed lines at |t| = 2.0): beta and investment are significant, while size, value, and profitability are not — a result that would surprise many textbook authors but is consistent with the 2014–2024 sample period.

If you are a quant researcher at a fund, this chart is the starting point for a conversation: "We can explain 15% of the cross-section with known factors. The remaining 85% is our opportunity. But most of it is noise. Week 4's ML models will try to separate the signal from the noise — using the feature matrix you just built in Deliverable 2 as their input."

---

## Summary of Discoveries

- **Factor replication quality is factor-dependent, not just data-dependent.** HML (value) replicates well from S&P 500 data (r ~ 0.83) because value-growth dispersion exists within large-caps. SMB (size) replicates poorly (r ~ 0.33) because the entire S&P 500 is large-cap. The relative ordering of replication quality (HML > MOM > RMW > SMB > CMA) tells you which characteristics have meaningful dispersion in your universe and which require broader data.

- **Construction methodology matters more than you think.** Switching from equal-weighted to value-weighted portfolios dropped MOM correlation from 0.85 to 0.64 — a 0.21 correlation gap from a single design choice. "The momentum factor" is not a unique object; it is an artifact of specific methodological decisions.

- **Static sorts introduce look-ahead bias.** Our HML correlation of ~0.83 is flattered by classifying stocks once using ~2025 fundamentals and applying those assignments backward over the full 2014–2024 window. A proper annual-rebalancing implementation would likely produce lower correlations. Always understand the methodology behind any benchmark comparison.

- **Feature engineering in finance requires domain knowledge.** P/E ratio — the most quoted valuation metric — is pathological for cross-sectional ML because it is undefined at E=0 and sign-flips for loss-makers. Earnings yield (E/P) is the correct feature. Winsorization at [1st, 99th] percentiles is insufficient for small universes; a z-cap at ±3 is the industry-standard safety net.

- **Factor models explain only 8–15% of cross-sectional variation.** The horse race shows FF5 at 15.3% R-squared — a clear winner, but a modest one. The average stock has ~4.9% monthly unexplained return under the best model. This is the residual alpha — and it is exactly what Week 4's ML models will attempt to capture.

- **Which factors are "priced" depends on the decade.** Beta is significantly priced in 2014–2024 (the AI/tech boom rewarded high-beta stocks), contradicting the classic "beta anomaly" documented with 1963–1990 data. Size and value are insignificant in our sample, despite being the star results in Fama and French (1992). Factor premia are sample-dependent — this is not a nuisance, it is the first-order problem.

- **Production data closes most of these gaps.** With CRSP/Compustat (4,000+ stocks, point-in-time fundamentals, proper annual rebalancing), factor replication correlations exceed 0.95, cross-sectional tests span 60+ years, and factor premia estimates are far more stable. The gaps you saw today are a tour of what free data costs you — and why institutional data infrastructure is a multi-billion-dollar industry.

- **The feature matrix you built is the direct input to Week 4.** Seven features, bounded within ±3, saved to Parquet. Next week's gradient boosting and neural network models will consume exactly this panel and ask: can ML extract signal from the 85% that linear factor models leave on the table?